In [1]:
from torchrecsys.external_datasets import Movielens_1M
from torchrecsys.datasets import Seq2SeqDataset
from torchrecsys.models.sequential import Bert4Rec
from pytorch_lightning import Trainer
import pandas as pd
import torch
from torch.utils.data import DataLoader

In [2]:
ratings, users, movies = Movielens_1M().load()

In [3]:
#Preprocess users
users['gender'], uniques = pd.factorize(users['gender'])
users['occupation'], uniques = pd.factorize(users['occupation'])
users['zip'], uniques = pd.factorize(users['zip'])
##Set category dtype
users['gender'] = users.gender.astype('category')
users['occupation'] = users.occupation.astype('category')
users['zip'] = users.zip.astype('category')

#Preprocess movies, 
##categories to index
movies['title'], uniques = pd.factorize(movies['title'])
movies['genres'], uniques = pd.factorize(movies['genres'])
##Set category dtype
movies['title'] = movies.title.astype('category')
movies['genres'] = movies.genres.astype('category')

max_item_id = movies['movie_id'].max()

In [4]:
sorted_sequences = ratings.sort_values(by=['timestamp']).groupby('user_id')['movie_id'].apply(list).to_frame()

In [7]:
dataset = Seq2SeqDataset(sorted_sequences, movies, sequence_id="movie_id", item_id="movie_id", max_item_id=max_item_id)
train = DataLoader(dataset, batch_size=512)

In [9]:
model = Bert4Rec(dataset.data_schema)

In [13]:
trainer = Trainer(max_epochs=1)
trainer.fit(model, train)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/jaimeferrando/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name               | Type             | Params
--------------------------------------------------------
0 | item_features      | ModuleList       | 33.5 K
1 | embedding          | BERTEmbedding    | 2.0 M 
2 | transformer_blocks | ModuleList       | 6.3 M 
3 | out                | Linear           | 2.0 M 
4 | criterion          | CrossEntropyLoss | 0     
--------------------------------------------------------
10.3 M    Trainable params
0         Non-trainable params
10.3 M    Total params
41.353    Total estimated model params size (MB)
/Users/jaimeferrando/mini

Training: 0it [00:00, ?it/s]

/Users/jaimeferrando/projects/github/torchrecsys/torchrecsys/datasets/datasets.py:201: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  torch.LongTensor(features),
/Users/jaimeferrando/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
